In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Alzheimers_Disease"
cohort = "GSE167559"

# Input paths
in_trait_dir = "../../input/GEO/Alzheimers_Disease"
in_cohort_dir = "../../input/GEO/Alzheimers_Disease/GSE167559"

# Output paths
out_data_file = "../../output/preprocess/Alzheimers_Disease/GSE167559.csv"
out_gene_data_file = "../../output/preprocess/Alzheimers_Disease/gene_data/GSE167559.csv"
out_clinical_data_file = "../../output/preprocess/Alzheimers_Disease/clinical_data/GSE167559.csv"
json_path = "../../output/preprocess/Alzheimers_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Dementia subtype prediction models constructed by penalized regression methods for multiclass classification using serum microRNA expression data"
!Series_summary	"There are many subtypes of dementia, and identification of diagnostic biomarkers that are minimally-invasive, low-cost, and efficient is desired. Circulating microRNAs (miRNAs) have recently gained attention as easily accessible and non-invasive biomarkers. We conducted a comprehensive miRNA expression analysis of serum samples from 1348 Japanese dementia patients, composed of four subtypes—Alzheimer’s disease (AD), vascular dementia, dementia with Lewy bodies (DLB), and normal pressure hydrocephalus—and 246 control subjects. We used this data to construct dementia subtype prediction models based on penalized regression models with the multiclass classification. We constructed a final prediction model using 46 miRNAs, which classified dementia patients from an independent validation set

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Optional, Callable, Dict, Any, List, Union

# 1. Determine gene expression data availability
# This dataset contains microRNA expression data, not gene expression data
is_gene_available = False  # miRNA data is not suitable for our gene expression analysis

# 2. Check for variable availability and define conversion functions

# 2.1 Trait (Alzheimer's Disease) data
# Looking at the sample characteristics, row 1 contains 'diagnosis: NPH'
# NPH stands for normal pressure hydrocephalus, which is not Alzheimer's Disease
# All samples are NPH, so this is a constant feature
trait_row = None  # All samples are NPH patients, not AD patients

# 2.2 Age data
# Row 2 contains age information
age_row = 2

def convert_age(age_str: str) -> Optional[float]:
    """Convert age string to float."""
    if not age_str or not isinstance(age_str, str):
        return None
    try:
        # Extract the value after the colon
        if ':' in age_str:
            age_value = age_str.split(':', 1)[1].strip()
            return float(age_value)
        else:
            return float(age_str.strip())
    except (ValueError, IndexError):
        return None

# 2.3 Gender data
# Row 3 contains gender information
gender_row = 3

def convert_gender(gender_str: str) -> Optional[int]:
    """Convert gender string to binary (0 for female, 1 for male)."""
    if not gender_str or not isinstance(gender_str, str):
        return None
    
    # Extract the value after the colon
    if ':' in gender_str:
        gender_value = gender_str.split(':', 1)[1].strip().lower()
    else:
        gender_value = gender_str.strip().lower()
    
    if 'female' in gender_value:
        return 0
    elif 'male' in gender_value:
        return 1
    else:
        return None

# Trait conversion function (even though trait data is not available)
def convert_trait(trait_str: str) -> Optional[int]:
    """Convert trait string to binary (0 for control, 1 for AD)."""
    if not trait_str or not isinstance(trait_str, str):
        return None
    
    # Extract the value after the colon
    if ':' in trait_str:
        value = trait_str.split(':', 1)[1].strip().lower()
    else:
        value = trait_str.strip().lower()
    
    # In this dataset, all samples are NPH, not AD
    if 'nph' in value:
        return 0  # Not AD
    elif 'ad' in value or 'alzheimer' in value:
        return 1  # AD
    else:
        return None

# 3. Save metadata about dataset usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical feature extraction
# We skip this step since trait_row is None (clinical data for our trait is not available)
# This dataset contains only NPH patients, not AD patients

False